In [1]:
%load_ext jupyter_black

In [2]:
import dataclasses
from abc import abstractmethod
from datetime import datetime, timedelta
from typing import Literal, Callable, TypeVar, ParamSpec, get_args

import s3fs
import pandas as pd
import xarray as xr
from IPython.display import HTML

from griblib._hrrr import ZArrTable

T = TypeVar("T")
P = ParamSpec("P")
idx: slice = pd.IndexSlice


class SharedTable:
    __table = ZArrTable[["vertical_level", "parameter_short_name"]]

    @property
    def __model_type(self):
        if isinstance(self, Forecast):
            return "fcst", "both"
        elif isinstance(self, Analysis):
            return "anl", "both"
        else:
            return "fcst", "anl", "both"

    def __repr__(self) -> str:
        return self.table.__repr__()

    def _repr_html_(self) -> HTML:
        return self.table._repr_html_()

    @property
    def table(self) -> pd.DataFrame:
        return self.__table.loc[idx[self.__model_type, :], :].droplevel(0).copy()


class Base(SharedTable):
    def __init__(self, hrrr: "HRRR2", level_type: Literal["sfc", "prs"]):
        model_type = "fcst" if isinstance(self, Forecast) else "anl"
        base_url = f"s3://hrrrzarr/{level_type}/" + hrrr.date_range.strftime(f"%Y%m%d/%Y%m%d_%Hz_{model_type}.zarr")

        def generate_urls():
            for long_name, (vlevel, short_name) in self.table.iterrows():
                base = base_url + f"/{vlevel}/{short_name}"
                yield (long_name, vlevel), tuple(zip(base, base + f"/{vlevel}"))

        self.hrrr = hrrr
        self._urldf = pd.DataFrame(dict(generate_urls()), index=hrrr.date_range)


def loadermethod(func: Callable[P, T]) -> Callable[P, T]:
    # possible_levels = get_args(func.__annotations__["vertical_level"])
    anno = func.__annotations__.copy()
    anno.pop("return", None)
    # ds_callback = xr.Dataset in func.__annotations__.values()
    default: tuple[str, ...] = func.__defaults__
    if default:
        (default_value,) = default

    long_name = func.__name__

    def inner(self: "Base", vertical_level: str = default_value):
        fs = self.hrrr.fs

        def generate():
            for urls in self._urldf[long_name, vertical_level]:
                yield xr.open_mfdataset((s3fs.S3Map(url, s3=fs) for url in urls), engine="zarr")

        return xr.concat(
            generate(),
            dim="valid_time",
            combine_attrs="override",
        )

    return inner


class Both(Base):
    @loadermethod
    @abstractmethod
    def temperature(
        self,
        # ds: xr.Dataset,
        /,
        vertical_level: Literal[
            "1000mb", "2m_above_ground", "500mb", "700mb", "850mb", "925mb", "surface"
        ] = "surface",
    ) -> pd.DataFrame:
        ...

        # return ds

    @loadermethod
    @abstractmethod
    def hail(
        self, vertical_level: Literal["0.1_sigma_layer", "entire_atmosphere", "surface"] = "surface"
    ) -> xr.Dataset:
        ...

    @loadermethod
    @abstractmethod
    def vertical_wind_shear(
        self, vertical_level: Literal["0.1_sigma_layer", "entire_atmosphere", "surface"] = "0.1_sigma_layer"
    ) -> xr.Dataset:
        ...


class Forecast(Both):
    ...


class Analysis(Both):
    ...


# LevelType enherits the SharedTable the so the entire table can be viewed
class LevelType(SharedTable):
    def __init__(self, hrrr: "HRRR2", level_type: Literal["sfc", "prs"]):
        self.__forecast = Forecast(hrrr, level_type)
        self.__analysis = Analysis(hrrr, level_type)

    @property
    def forecast(self):
        return self.__forecast

    @property
    def analysis(self):
        return self.__analysis


@dataclasses.dataclass
class HRRR2:
    start_date: datetime
    hours: int
    date_range: pd.DatetimeIndex
    fs: s3fs.S3FileSystem

    @property
    def surface(self):
        return LevelType(self, level_type="sfc")

    @property
    def pressure(self):
        return LevelType(self, level_type="prs")


def load_hrrr(start_date: datetime, hour_delta: int) -> HRRR2:
    date_range = pd.date_range(start_date, start_date + timedelta(hours=hour_delta))
    return HRRR2(start_date, hour_delta, date_range, s3fs.S3FileSystem(anon=True))


hrrr = load_hrrr(datetime(2022, 6, 16, 6), 5)

In [3]:
# uses surface by default
hrrr.surface.forecast.temperature()

<xarray.Dataset>
Dimensions:                  (time: 48, projection_x_coordinate: 1799,
                              projection_y_coordinate: 1059, valid_time: 1)
Coordinates:
  * time                     (time) datetime64[ns] 2022-06-16T07:00:00 ... 20...
  * projection_x_coordinate  (projection_x_coordinate) float64 -2.698e+06 ......
  * projection_y_coordinate  (projection_y_coordinate) float64 -1.587e+06 ......
Dimensions without coordinates: valid_time
Data variables:
    TMP                      (valid_time, time, projection_y_coordinate, projection_x_coordinate) float16 dask.array<chunksize=(1, 48, 150, 150), meta=np.ndarray>
    forecast_period          (valid_time, time) timedelta64[ns] dask.array<chunksize=(1, 48), meta=np.ndarray>
    forecast_reference_time  (valid_time) datetime64[ns] 2022-06-16T06:00:00

In [4]:
hrrr.surface.forecast.temperature(vertical_level="1000mb")

<xarray.Dataset>
Dimensions:                  (time: 48, projection_x_coordinate: 1799,
                              projection_y_coordinate: 1059, valid_time: 1)
Coordinates:
  * time                     (time) datetime64[ns] 2022-06-16T07:00:00 ... 20...
  * projection_x_coordinate  (projection_x_coordinate) float64 -2.698e+06 ......
  * projection_y_coordinate  (projection_y_coordinate) float64 -1.587e+06 ......
Dimensions without coordinates: valid_time
Data variables:
    TMP                      (valid_time, time, projection_y_coordinate, projection_x_coordinate) float16 dask.array<chunksize=(1, 48, 150, 150), meta=np.ndarray>
    forecast_period          (valid_time, time) timedelta64[ns] dask.array<chunksize=(1, 48), meta=np.ndarray>
    forecast_reference_time  (valid_time) datetime64[ns] 2022-06-16T06:00:00

In [5]:
hrrr.surface.forecast.hail(vertical_level="entire_atmosphere")

<xarray.Dataset>
Dimensions:                  (time: 48, projection_x_coordinate: 1799,
                              projection_y_coordinate: 1059, valid_time: 1)
Coordinates:
  * time                     (time) datetime64[ns] 2022-06-16T07:00:00 ... 20...
  * projection_x_coordinate  (projection_x_coordinate) float64 -2.698e+06 ......
  * projection_y_coordinate  (projection_y_coordinate) float64 -1.587e+06 ......
Dimensions without coordinates: valid_time
Data variables:
    HAIL_1hr_max_fcst        (valid_time, time, projection_y_coordinate, projection_x_coordinate) float16 dask.array<chunksize=(1, 48, 150, 150), meta=np.ndarray>
    forecast_period          (valid_time, time) timedelta64[ns] dask.array<chunksize=(1, 48), meta=np.ndarray>
    forecast_reference_time  (valid_time) datetime64[ns] 2022-06-16T06:00:00

In [6]:
hrrr.surface.forecast.hail(vertical_level="surface")

<xarray.Dataset>
Dimensions:                  (time: 48, projection_x_coordinate: 1799,
                              projection_y_coordinate: 1059, valid_time: 1)
Coordinates:
  * time                     (time) datetime64[ns] 2022-06-16T07:00:00 ... 20...
  * projection_x_coordinate  (projection_x_coordinate) float64 -2.698e+06 ......
  * projection_y_coordinate  (projection_y_coordinate) float64 -1.587e+06 ......
Dimensions without coordinates: valid_time
Data variables:
    HAIL_1hr_max_fcst        (valid_time, time, projection_y_coordinate, projection_x_coordinate) float16 dask.array<chunksize=(1, 48, 150, 150), meta=np.ndarray>
    forecast_period          (valid_time, time) timedelta64[ns] dask.array<chunksize=(1, 48), meta=np.ndarray>
    forecast_reference_time  (valid_time) datetime64[ns] 2022-06-16T06:00:00